# Part 1: Deployment and Data Upload

In [1]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 24579932160 (22.89 GB)
DFS Remaining: 24579883008 (22.89 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.4:9866 (project-4-perrito_p4-dn-2.project-4-perrito_p4_default)
Hostname: 352011038fb0
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 13514309632 (12.59 GB)
DFS Remaining: 12289941504 (11.45 GB)
DFS Used%: 0.00%
DFS Remaining%: 47.60

In [2]:
import requests

url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
r = requests.get(url, allow_redirects=True)

open('hdma-wi-2021.csv', 'wb').write(r.content)

174944099

In [3]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [11]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

cp: `hdfs://boss:9000/single.csv': File exists
cp: `hdfs://boss:9000/double.csv': File exists


In [12]:
!hdfs dfs -ls hdfs://boss:9000/

Found 2 items
-rw-r--r--   2 root supergroup  174944099 2023-10-26 23:10 hdfs://boss:9000/double.csv
-rw-r--r--   1 root supergroup  174944099 2023-10-26 23:10 hdfs://boss:9000/single.csv


In [13]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


# Part 2: WebHDFS

In [14]:
#q3
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.json()

{'FileStatus': {'accessTime': 1698361836305,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698361843164,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [15]:
#q4
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.json()["Location"]

'http://352011038fb0:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [16]:
#q5
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
containerDict = {}
for i in range(len(r.json().get("BlockLocations").get("BlockLocation"))):
    id = r.json().get("BlockLocations").get("BlockLocation")[i].get("hosts")[0]
    blocks = r.json().get("BlockLocations").get("BlockLocation")[i].get("length") / 1048576
    count = containerDict.setdefault(id, 0)
    if (count == None):
            count = 0
    containerDict.update({id: (count+blocks)});
containerDict

{'352011038fb0': 85.0, '4e41b8f42293': 81.83969402313232}

# Part 3: PyArrow

In [10]:
import pyarrow as pa
import pyarrow.fs

In [11]:
#q6
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
retval = None
with hdfs.open_input_file("/single.csv") as f:
    retval = f.read_at(10,0)

retval

2023-10-26 18:52:32,073 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [12]:
import io
import re

In [13]:
#q7
count = 0
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for i, line in enumerate(reader):
        if re.search(r'Single Family', line):
            count += 1
count

444874